# Interaction Network

This jupyter notebook is for implementing a paper [Interaction Networks for Learning about Objects, Relations and Physics](https://arxiv.org/abs/1612.00222) of [Deepmind](https://deepmind.com/). This code will be implemented by [Tensorflow](https://www.tensorflow.org/)

## Import Libraries

In [1]:
import numpy as np 
import scipy.io
import matplotlib.pyplot as plt
import tensorflow as tf 

C:\Users\Q\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Parameters

In [2]:
# Parameters of the data
Len_state = 6 # Length of the state
Len_relation = 1 # Length of the relation 
Len_external = 1 # Length of the external effect
Len_effect = 50 # Length of the effect
Len_output = 4 # Length of the output (x position, y position, x velocity y velocity)

Num_obj = 6 # Number of the objects 
Num_relation = 30 # Number of the relations

B_shape = [(2*Len_state) + Len_relation, Num_relation]
C_shape = [(Len_state + Len_external + Len_effect), Num_obj]

# Parameter for training
Num_batch = 512
Num_epoch = 500
Learning_rate = 0.001

Is_train = True
GPU_fraction = 0.3

# Parameter for network
fR_dense1 = [B_shape[0], 512]
fR_dense2 = [512, 1024]
fR_dense3 = [1024, Len_effect]

fO_dense1 = [C_shape[0], 512]
fO_dense2 = [512, 1024]
fO_dense3 = [1024, Len_output]

## Get data

In [3]:
dataset_train = scipy.io.loadmat('./data/Training_dataset.mat')
dataset_test  = scipy.io.loadmat('./data/Testing_dataset.mat')

data_x = np.float32(dataset_train['X'])
data_y = np.float32(dataset_train['Y'])

data_x_test = dataset_test['X']
data_y_test = dataset_test['Y']

# data = np.concatenate([data_x, data_y], axis = 1)
# np.random.shuffle(data)

print("Data_x shape: " + str(data_x.shape))
print("Data_y shape: " + str(data_y.shape))
print("TestData_x shape: " + str(data_x_test.shape))
print("TestData_y shape: " + str(data_y_test.shape))

Data_x shape: (1000000, 6, 6)
Data_y shape: (1000000, 6, 6)
TestData_x shape: (100000, 6, 6)
TestData_y shape: (100000, 6, 6)


## Functions

In [4]:
# Initialize weights and bias 
def weight_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

def bias_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

## Interaction Network

In [5]:
tf.reset_default_graph()

O  = tf.placeholder(tf.float32, shape=[None, Len_state, Num_obj])
Rr = tf.placeholder(tf.float32, shape=[None, Num_obj, Num_relation])
Rs = tf.placeholder(tf.float32, shape=[None, Num_obj, Num_relation])
Ra = tf.placeholder(tf.float32, shape=[None, Len_relation, Num_relation])
X  = tf.placeholder(tf.float32, shape=[None, Len_external, Num_obj])

# B -> input of the relation-centric function f_R
B = tf.concat([tf.matmul(O,Rr),tf.matmul(O,Rs),Ra], axis = 1)

# Define weight and bias of fR
with tf.variable_scope('fR_'):
    fR_w_fc1 = weight_variable('w_fc1', fR_dense1)
    fR_w_fc2 = weight_variable('w_fc2', fR_dense2)
    fR_w_fc3 = weight_variable('w_fc3', fR_dense3)

    fR_b_fc1 = bias_variable('b_fc1', fR_dense1[1])
    fR_b_fc2 = bias_variable('b_fc2', fR_dense2[1])
    fR_b_fc3 = bias_variable('b_fc3', fR_dense3[1])

#list of the effects
e_list = []

# fully connected for each column to obtain effect e
for i in range(Num_relation):
    b = tf.reshape(tf.slice(B,[0,0,i],[-1,-1,1]),[-1, B_shape[0]])

    h_fc1 = tf.nn.relu(tf.matmul(b, fR_w_fc1)+fR_b_fc1)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, fR_w_fc2)+fR_b_fc2)

    e = tf.nn.relu(tf.matmul(h_fc2, fR_w_fc3)+fR_b_fc3)
    e_list.append(e)

# Effect Matrix E
E = tf.stack(e_list, axis = 2)
E_bar = tf.matmul(E, Rr, transpose_b = True)

# C -> input of the object-centric function f_O
C = tf.concat([O, X, E_bar], axis = 1)

# Define weight and bias of fO
with tf.variable_scope('fO_'):
    fO_w_fc1 = weight_variable('w_fc1', fO_dense1)
    fO_w_fc2 = weight_variable('w_fc2', fO_dense2)
    fO_w_fc3 = weight_variable('w_fc3', fO_dense3)

    fO_b_fc1 = bias_variable('b_fc1', fO_dense1[1])
    fO_b_fc2 = bias_variable('b_fc2', fO_dense2[1])
    fO_b_fc3 = bias_variable('b_fc3', fO_dense3[1])

#list of the outputs
p_list = []

# fully connected for each column to obtain output p
for i in range(Num_obj):
    c = tf.reshape(tf.slice(C,[0,0,i],[-1,-1,1]),[-1, C_shape[0]])

    fO_h_fc1 = tf.nn.relu(tf.matmul(c, fO_w_fc1)+fO_b_fc1)
    fO_h_fc2 = tf.nn.relu(tf.matmul(fO_h_fc1, fO_w_fc2)+fO_b_fc2)

    p = tf.nn.relu(tf.matmul(fO_h_fc2, fO_w_fc3)+fO_b_fc3)
    p_list.append(p)

# Output matrix P
P = tf.stack(p_list, axis = 2)

## Loss and Train

In [6]:
# Label (next state data)
y = tf.placeholder(tf.float32, [None, Len_output, Num_obj])

# Loss
loss = tf.reduce_mean(tf.reduce_sum(tf.reduce_mean(tf.square(y - P), axis = 2), axis = 1))

# Training step
train_step = tf.train.AdamOptimizer(learning_rate = Learning_rate, epsilon = 1e-02).minimize(loss)

## Initialize session

In [7]:
# Initialize variables
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = GPU_fraction

sess = tf.InteractiveSession(config=config)

init = tf.global_variables_initializer()
sess.run(init)

# Load the file if the saved file exists
saver = tf.train.Saver()

if Is_train == False:
    # Restore variables from disk.
    saver.restore(sess, "./saved_network/model.ckpt")
    print("Model restored.")

## Training

In [ ]:
if Is_train == True:
    train_data_num = data_x.shape[0]

    for i in range(Num_epoch):
        # Making batches
        random_idx = np.arange(train_data_num)
        np.random.shuffle(random_idx)
        
        batch_count = 1
        num_batch_data = 0
    
        for j in range(0, train_data_num, Num_batch):
            if j + Num_batch < train_data_num:
                batch_index = [j, j + Num_batch]
            else:
                batch_index = [j, train_data_num-1]
            
            batch_x = data_x[random_idx[batch_index[0]:batch_index[-1]],:,:]
            batch_y = data_y[random_idx[batch_index[0]:batch_index[-1]], 0:4, :]
            
            # Set relation and external -> every objects are each other's sender and receiver
            batch_Rr = np.zeros([batch_index[1]-batch_index[0], Num_obj, Num_relation], dtype = np.float32)
            batch_Rs = np.zeros([batch_index[1]-batch_index[0], Num_obj, Num_relation], dtype = np.float32)
            batch_Ra = np.zeros([batch_index[1]-batch_index[0], Len_relation, Num_relation], dtype = np.float32)
        
            batch_external = np.zeros([batch_index[1]-batch_index[0], Len_external, Num_obj], dtype = np.float32)
            
            relation_idx = 0
            for m in range(Num_obj):
                for n in range(Num_obj):
                    if m != n:
                        batch_Rs[:,m,relation_idx] = 1
                        batch_Rr[:,n,relation_idx] = 1
                        
                        relation_idx = relation_idx + 1         
           
            # Train
            _, Loss_train = sess.run([train_step, loss], feed_dict={O: batch_x, Rr: batch_Rr, Rs: batch_Rs, Ra: batch_Ra, X: batch_external, y: batch_y})
            
            print("Batch: " + str(j) + '/' + str(train_data_num), end="\r")
    
        # Print Progress
        print("Epoch: " + str(i+1) + ' / ' +
              "Loss: " + str(Loss_train)) 

    save_path = saver.save(sess, 'saved_networks/model.ckpt')
    print("Model saved in file: %s" % save_path)
        

Epoch: 1 / Batch: 999936/1000000 / Loss: 0.55036837
Epoch: 2 / Batch: 999936/1000000 / Loss: 0.57091445
Epoch: 3 / Batch: 999936/1000000 / Loss: 0.59065205
Epoch: 4 / Batch: 999936/1000000 / Loss: 0.58040714
Epoch: 5 / Batch: 999936/1000000 / Loss: 0.58295023
Epoch: 6 / Batch: 999936/1000000 / Loss: 0.5669404
Epoch: 7 / Batch: 999936/1000000 / Loss: 0.5515185
Epoch: 8 / Batch: 999936/1000000 / Loss: 0.5791697
Epoch: 9 / Batch: 999936/1000000 / Loss: 0.54065186
Epoch: 10 / Batch: 999936/1000000 / Loss: 0.56858593
Epoch: 11 / Batch: 999936/1000000 / Loss: 0.52866524
